In [1]:
!pip install torch transformers accelerate bitsandbytes peft

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 18.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 17.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 17.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 13.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 8.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 4.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 45.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.0/67.0 MB 7.0 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalling

In [2]:
# In a new cell in your Google Colab notebook

# --- Step 1: Access the Secret Token ---
# This securely loads the token you stored in the Secrets Manager.
from google.colab import userdata
hf_token = userdata.get('HF_TOKEN')

# --- Step 2: Import necessary libraries ---
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer
from peft import PeftModel

# --- Configuration ---
base_model_id = "meta-llama/Llama-3.1-8B"
adapter_id = "SidMcStarter/legal-llama-adapters"
output_dir = "merged_legal_llama"

# --- Main Script ---
print(f"Loading base model: {base_model_id}")
# Load the base model, passing the token directly for authentication.
base_model = AutoModelForCausalLM.from_pretrained(
    base_model_id,
    load_in_4bit=True,
    torch_dtype=torch.bfloat16,
    device_map="auto",
    token=hf_token  # <--- Use the token here
)

tokenizer = AutoTokenizer.from_pretrained(
    base_model_id,
    token=hf_token  # <--- And also here
)

print(f"\nLoading LoRA adapters from: {adapter_id}")
# Apply the adapters to the base model.
model = PeftModel.from_pretrained(
    base_model,
    adapter_id,
    token=hf_token  # <--- And here for the adapters
)

print("\nMerging adapters into the base model...")
# Merge the LoRA layers into the base model.
merged_model = model.merge_and_unload()
print("Merge complete.")

print(f"\nSaving the final merged model to: {output_dir}")
# Save the complete, merged model to the Colab filesystem.
merged_model.save_pretrained(output_dir)
tokenizer.save_pretrained(output_dir)

print("\n--------------------------------------------------")
print(f"✅ Success! Your merged model is saved in the '{output_dir}' directory.")
print("This directory is now ready for the GGUF conversion process.")
print("--------------------------------------------------")

Loading base model: meta-llama/Llama-3.1-8B


config.json:   0%|          | 0.00/826 [00:00<?, ?B/s]

The `load_in_4bit` and `load_in_8bit` arguments are deprecated and will be removed in the future versions. Please, pass a `BitsAndBytesConfig` object in `quantization_config` argument instead.
None of the available devices `available_devices = None` are supported by the bitsandbytes version you have installed: `bnb_supported_devices = {'xpu', 'cuda', 'mps', 'npu', 'hpu', '"cpu" (needs an Intel CPU and intel_extension_for_pytorch installed and compatible with the PyTorch version)'}`. Please check the docs to see if the backend you intend to use is available and how to install it: https://huggingface.co/docs/bitsandbytes/main/en/installation#multi-backend


RuntimeError: None of the available devices `available_devices = None` are supported by the bitsandbytes version you have installed: `bnb_supported_devices = {'xpu', 'cuda', 'mps', 'npu', 'hpu', '"cpu" (needs an Intel CPU and intel_extension_for_pytorch installed and compatible with the PyTorch version)'}`. Please check the docs to see if the backend you intend to use is available and how to install it: https://huggingface.co/docs/bitsandbytes/main/en/installation#multi-backend